# Tax optimisation

## Tax slabs

First, we summarise the current taxation scheme for [corporations](https://cleartax.in/s/corporate-tax) and [individuals](https://cleartax.in/s/income-tax-slabs/) in FY2020-21.

Each scheme comprises a set of slabs and the applicable tax rate to income within that slab, as well as a similar set of surcharges applicable to income above a certain threshold.

The corporate tax scheme described here applies to companies with an annual turnover not exceeding 4B (400 Cr) that have not opted for the new taxation regime u/s [115BAA](https://cleartax.in/s/section-115-baa-tax-rate-domestic-companies/) (for all domestic companies) or [115BAB](https://cleartax.in/s/section-115-bab-corporate-tax-rate-new-manufacturing-companies/) (for manufacturing companies).

The individual scheme represents the tax slabs for an individual below 60 years of age under the "old regime".

In [1]:
corporate_taxes = {
    "slabs": [
        (0, float('inf'), 0.25),
    ],
    "surcharges": [
        (10_000_000, 100_000_000, 0.07),
        (100_000_001, float('inf'), 0.12),
    ],
}

individual_taxes = {
    "slabs": [
        (0, 250_000, 0),
        (250_001, 500_000, 0.05),
        (500_001, 1_000_000, 0.20),
        (1_000_001, float('inf'), 0.30),
    ],
    "surcharges": [
        (5_000_000, 10_000_000, 0.10),
        (10_000_001, float('inf'), 0.15),
    ],
    "rebates": [
        # u/s 87A
        (0, 500_000, 12_500)
    ]
}

While we're at it, let's define the opt-in "new regime" for individuals (with lower rates, but almost no permissible deductions) as a separate scheme. Note that this one doesn't vary with age (whereas the old regime relaxes the slab boundaries for senior citizens).

In [2]:
individual_new_regime_taxes = {
    "slabs": [
        (0, 250_000, 0),
        (250_001, 300_000, 0.05),
        (300_001, 500_000, 0.05),
        (500_001, 750_000, 0.10),
        (750_001, 1_000_000, 0.15),
        (1_000_001, 1_250_000, 0.20),
        (1_250_001, 1_500_000, 0.25),
        (1_500_001, float('inf'), 0.30),
    ],
    "surcharges": [
        (5_000_000, 10_000_000, 0.10),
        (10_000_001, float('inf'), 0.15),
    ],
    "rebates": [
        # u/s 87A
        (0, 500_000, 12_500)
    ]
}

## Income tax computation

Next, we define a function to calculate the tax payable for a given `amount` of taxable income according to a particular `scheme` as defined above.

Note that this function doesn't know anything about any permissible deductions—you must first apply all deductions to the gross income to arrive at the taxable income. It calculates the [marginal relief](https://cleartax.in/s/marginal-relief-surcharge/) available for the surcharge computation, as well as the rebate available to individuals in the lowest tax slab u/s [87A](https://cleartax.in/s/income-tax-rebate-us-87a/).

In [7]:
def income_tax(amount, scheme):
    tax = 0

    # First, we must apply the slab rate for the relevant portion of each
    # applicable slab.
    for slab in scheme['slabs']:
        max = slab[1]
        if amount < max:
            max = amount
        if amount > slab[0]:
            tax += (max - slab[0]) * slab[2]

    # Next, we calculate the surcharge, if applicable.
    #
    # Note that there can be only one surcharge, unlike the slab computation
    # above (where taxes are added for every slab below the taxable amount).
    #
    # When we calculate the surcharge, we must also check if "marginal relief"
    # is applicable, whereby the surcharge is reduced so that a higher taxable
    # amount doesn't result in a lower post-tax profit.
    for slab in scheme['surcharges']:
        if slab[0] <= amount <= slab[1]:
            surcharge = tax * slab[2]
            excess_income = amount - slab[0]
            threshold_tax = income_tax(slab[0]-1, scheme)/1.04
            excess_tax = tax + surcharge - threshold_tax
            relief = excess_tax - excess_income
            if relief > 0:
                surcharge -= relief

            tax += surcharge
            break

    # Handle any applicable rebates (e.g., u/s 87A)
    for r in scheme.get('rebates', []):
        if r[0] <= amount <= r[1]:
            rebate = r[2]
            if tax < rebate:
                rebate = tax
            tax -= rebate

    # Finally, we apply the 4% health and education cess.
    tax *= 1.04

    return tax

With this function, we can now compare the income tax payable by a company or an individual for a given amount (of taxable income, not gross income).

In [9]:
import pandas as pd
from IPython.display import HTML, display
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from matplotlib import pyplot as plt

%matplotlib inline

def format_tax(amount, scheme):
    return format(income_tax(amount, scheme), ',.2f')

def tax_table(n):
    taxes = [
        ["Corporate", format_tax(n, corporate_taxes)],
        ["Individual", format_tax(n, individual_taxes)],
    ]
    display(HTML(pd.DataFrame(taxes, columns=["Taxpayer", "Payable"]).to_html()))

slider = interactive(tax_table, n=widgets.IntSlider(min=0, max=100_000_000, step=100_000, value=100_000, description='Taxable', readout_format=',d'))
    
output = widgets.Output()
with output:
    plt.title("Tax comparison")
    plt.xlabel("Taxable amount")
    plt.ylabel("Tax")

    amounts = []
    corporate = []
    individual = []
    for n in range(0, 15_000_000, 50_000):
        amounts.append(n)
        corporate.append(income_tax(n, corporate_taxes))
        individual.append(income_tax(n, individual_taxes))
        
    c_line , = plt.plot(amounts, corporate, label='Corporate')
    i_line , = plt.plot(amounts, individual, label='Individual')
    plt.legend(handles=[c_line, i_line])
    
    xticks = []
    xlabels = []
    for n in range(0, 15_000_000, 2_500_000):
        xticks.append(n)
        if n < 10_000_000:
            xlabels.append('%sL' % int(n/100_000))
        else:
            xlabels.append('%sCr' % format(n/10_000_000, '.2f'))
    plt.xticks(ticks=xticks, labels=xlabels)
    
    yticks = []
    ylabels = []
    for n in range(0, 5_000_000, 1_000_000):
        yticks.append(n)
        ylabels.append('%sL' % int(n/100_000))
    plt.yticks(ticks=yticks, labels=ylabels)
    plt.show()

ui = widgets.HBox([slider, output])
display(ui)